In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/credit-card-customers/BankChurners.csv')
df = df.drop(['Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2','Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1'],axis =1)
df.head()

So we observe that there are a few categorical features such as Gender,Education Level, Marital Status, Income Category and Card Category

We will explore these against the possible attrition across to understand any underlying themes governing attrition for hypothesis formulation



In [ ]:
x = df.groupby('Attrition_Flag')['CLIENTNUM'].count()
#print(x)
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.graph_objects as go
plt.pie(df.groupby('Attrition_Flag')['CLIENTNUM'].count().sort_values(ascending= False)
,autopct = '%.1f%%',shadow = True,colors = ['lightblue','lightsteelblue'],labels = df['Attrition_Flag'].unique())
plt.show()

So on a macro level, there were around 16% Card Members who have cancelled their membership from the entire customer line.

I would also like to understand if there were any card members who had multiple cards and any possible instances of cancelling the lower or higher end cards in that case assuming the customers feel they no longer need it.


In [ ]:
total_records = df['CLIENTNUM'].count()
unique_CM = df['CLIENTNUM'].nunique()
Duplicates = total_records - unique_CM
print(f"There are {Duplicates} duplicate Card Members in the dataset")

Since we do not have any CMs with multiple Cards, we can rule out the hypothesis regarding the CMs cancelling either of the cards as we had assumed earlier

# **Exploratory Analysis **

Lets Create some exploratory plots for understanding the categorical metrics against the Attrition/Churn 

In [ ]:
def create_chart(indicator):
    df1 = df.groupby(['Attrition_Flag',indicator])['CLIENTNUM'].count().reset_index()
    #print(df1)
    df2 = df.groupby(indicator)['CLIENTNUM'].count().reset_index()
    #print(df2)
    df3 = pd.merge(df1,df2,on = indicator)
    #print(df3)
    df3['Percentage_Attrition'] = df3['CLIENTNUM_x']/df3['CLIENTNUM_y']
    fig = px.bar(df3, x = indicator,y = 'Percentage_Attrition', color = 'Attrition_Flag' , barmode = 'group', color_continuous_scale= 'teal')
    fig.show()
create_chart('Card_Category')

So from the above chart, we can infer that Platinum card*(highest Variant) has highest attrition rate among the various Card types. 

A hypothesis around this could be that the Platinum fees are higher and the churned CM's might have found it hard to continue their membership at those rates or might not be using it for some months

* Assuming Blue < Silver < Gold < Platinum

In [ ]:
create_chart('Gender')

It can be observed that on average there are ~3 more female CM's amongh 100 CMs that are lost as customers by the bank.


In [ ]:
create_chart('Marital_Status')

In [ ]:
create_chart('Education_Level')

We see an interesting trend here, CM's who are highly educated (Postgrads and Doctorates) have higher attrition rates. Lets explore it further 

In [ ]:
create_chart('Income_Category')

In [ ]:
#print(df.columns)

def create_scatter(df,x,y):
    sns.scatterplot(data=df,x = x,y = y,hue='Attrition_Flag')
    

create_scatter(df,'Total_Trans_Amt','Total_Trans_Ct')


In [ ]:
sns.scatterplot(data=df,x='Total_Trans_Amt',y='Total_Trans_Ct',hue='Attrition_Flag')

In [ ]:
create_scatter(df,'Total_Trans_Amt','Total_Revolving_Bal')

In [ ]:
create_scatter(df,'Total_Amt_Chng_Q4_Q1','Total_Ct_Chng_Q4_Q1')


It can be observed that almost all customers with large transaction amount change from Q4 to Q1 with a similar incremental increase in transaction count are less likely to cancel the Card/Product

I would like to explore if there is a specific Card Category that is responsible for the churn of customers since I am of the view that a specific Card category catering with which customers aren't happy with in terms of Credit Limit and offers are at risk of attrition.



In [ ]:
x = sns.FacetGrid(data = df ,col = 'Card_Category', hue = 'Attrition_Flag')
x.map(sns.scatterplot,'Total_Trans_Amt','Total_Trans_Ct')